# This code uses visualization tools to viz your suite2p results!

Open terminal:

    conda activate suite2p
    jupyter-lab

Open Suite2p_Run and work through the notebook. Wait until algorithm is complete.

Open a separate terminal:

    conda activate suite2p
    suite2p

Now when in suite2p, select:
1) mean img (enhanced) or max projection

Toggle back to jupyter-notebook and open `Suite2p_quickViz` to see your components

Drag your cursor to see your components. If you find any components that are not caught by suite2p, go back to suite2p and:
1) File > Manual Labeling
2) mean img (enhanced)
3) Press "add ROI"
4) Find the ROI and create boundaries
5) Press "Extract ROIs"
6) Press "Save and Quit"
7) Go back to `Suite2p_quickViz` and watch the results!

When you're done, open `Suite2p_exploreResults.ipynb`

In [3]:
from pathlib import Path
from tempfile import TemporaryDirectory
import matplotlib.pyplot as plt
import numpy as np
import os
import tifffile
import xmltodict
import suite2p
import fastplotlib as fpl

# Figure Style settings for notebook.
import matplotlib as mpl
mpl.rcParams.update({
    'axes.spines.left': False,
    'axes.spines.bottom': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'legend.frameon': False,
    'figure.subplot.wspace': .01,
    'figure.subplot.hspace': .01,
    'figure.figsize': (9, 9),
    'ytick.major.left': False,
})
jet = mpl.cm.get_cmap('jet')
jet.set_bad(color='k')

import os
import sys
root_dir = os.path.split(os.path.split(os.getcwd())[0])[0] # get root
utils_dir = os.path.join(root_dir,'code','utils') # get utils folder path
sys.path.append(utils_dir) # add it to system path (not ideal) - doing this to reduce pip installs for local lab usage
import plot_tools as fpl_tools # import movie

/var/folders/f3/11w5x6bs32gd1xc4lh41qdnr0000gp/T/ipykernel_1285/2855019485.py:24: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jet = mpl.cm.get_cmap('jet')


# set `output_path` to suite2p output and watch the fastplotlib plot in action!

In [4]:
output_path = r"/Users/js0403/2p/A04/suite2p/plane0"
fpl_object = fpl_tools.play_suite2p_movie(suite2p_path=output_path)
fpl_object.play_movie_plot_mask(components_type='both',plot_type='double')

/Users/js0403/2p/A04/img.tif


RFBOutputContext()

In [3]:
# movies and associated frame rates
#output_path = r"/Users/js0403/slice_imaging/AG4 data/suite2p/plane0"
output_path = r"/Users/js0403/2p/A04/suite2p/plane0"

# list the files in the directory
listed_data = os.listdir(output_path)

# loads in your data and automatically assigns their inputs
var_names =[]
for i in listed_data:
    if '.npy' in i:
        if 'ops' in i:
            globals()[i.split('.')[0]]=np.load(os.path.join(output_path,i),allow_pickle=True).item()
            pass
        else:
            globals()[i.split('.')[0]]=np.load(os.path.join(output_path,i),allow_pickle=True)
        var_names.append([i.split('.')[0]])

# define data_path
if len(ops['data_path'])>1 or len(ops['tiff_list'])>1:
    print("This code does not support multi-file registration")
else:
    data_path = os.path.join(ops['data_path'][0],ops['tiff_list'][0])
    print(data_path)

# prep stuff
output_op_file = np.load(Path(ops['save_path']).joinpath('ops.npy'), allow_pickle=True).item()
output_op_file.keys() == ops.keys()
stats_file = Path(ops['save_path']).joinpath('stat.npy')
iscell = np.load(Path(ops['save_path']).joinpath('iscell.npy'), allow_pickle=True)[:, 0].astype(bool)
stats = np.load(stats_file, allow_pickle=True)
stats.shape, iscell.shape

# get image object
im = suite2p.ROI.stats_dicts_to_3d_array(stats, Ly=ops['Ly'], Lx=ops['Lx'], label_id=True)
im[im == 0] = np.nan

# lazy load video
images = tifffile.memmap(data_path)

# cell rois
cell_roi = np.nanmax(im[iscell], axis=0)
cell_roi.shape
cell_roi[~np.isnan(cell_roi)]=20

#%% Watch the movie using fastplotlib
if 'iw_movie' in locals():
    iw_movie.close()

iw_movie = fpl.ImageWidget(
    data=images,
    slider_dims=["t"],
    cmap="gray"
)

# add a column to overlay functional activity on structural video
struct_rgba = np.zeros((images.shape[1], images.shape[2], 4), dtype=np.float32)
struct_rgba[:, :, 1] = cell_roi
struct_rgba[..., -1] = 10
iw_movie.gridplot[0,0].add_image(struct_rgba, name='struct')
iw_movie.gridplot[0,0]
iw_movie.gridplot[0,0]["struct"].data[..., -1] = .3
#iw_movie.gridplot[0,0].add_image(cell_roi, name='struct')

# visualize the movie
iw_movie.show(sidecar=True)

/Users/js0403/slice_imaging/AG4 data/ExpFit_BleachCorr_Process_111.vsi - Acquisition_spatDownSampledx3.tif


/var/folders/f3/11w5x6bs32gd1xc4lh41qdnr0000gp/T/ipykernel_82460/3186610997.py:41: RuntimeWarning: All-NaN slice encountered
  cell_roi = np.nanmax(im[iscell], axis=0)


RFBOutputContext()

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='auto-scale scene'), Button(icon='align-center', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='auto-center scene'), ToggleButton(value=True, icon='hand-pointer', layout=Layout(width='auto'), tooltip='panzoom controller'), ToggleButton(value=True, description='1:1', layout=Layout(width='auto'), style=ToggleButtonStyle(font_weight='bold'), tooltip='maintain aspect'), Button(icon='arrow-down', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='y-axis direction'), Button(icon='draw-polygon', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='add PolygonSelector'), ToggleButton(value=False, icon='video', layout=Layout(width='auto'), tooltip='record'), Dropdown(description='Subplots:', layout=Layout(width='200px'), options=('(0, 0)',), value='(0, 0)'))), IpywidgetImageWidgetToolbar(children=(HBox